In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from bs4 import BeautifulSoup
import re
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer

/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
df = pd.read_csv('labeledTrainData.tsv', delimiter='\t')
df2 = pd.read_csv('unlabeledTrainData.tsv', delimiter='\t', error_bad_lines= False)
df_test = pd.read_csv('testData.tsv', delimiter= '\t')

Skipping line 43043: expected 2 fields, saw 3



In [3]:
english_stopwords = set(nltk.corpus.stopwords.words('english'))
def review_to_words(txt):
    # comment following to speed up.
    # bs = BeautifulSoup(txt)
    # txt = re.sub('[^a-zA-Z]', ' ', bs.get_text())
    words = filter(lambda x: x and x.isalpha() and len(x) >= 3 and x not in english_stopwords, txt.lower().split())
    # words = filter(lambda x: x and x not in english_stopwords, txt.lower().split())
    # return ' '.join(words) # for BOW
    return words # for word2vec

In [4]:
df['clean_review'] = df['review'].apply(review_to_words)
df2['clean_review'] = df2['review'].apply(review_to_words)
df_test['clean_review'] = df_test['review'].apply(review_to_words)

In [5]:
clean_reviewes = pd.Series.append(df['clean_review'], df2['clean_review'])
clean_reviewes = clean_reviewes.reset_index(drop = True)

In [229]:
tv = TfidfVectorizer(min_df = 3, max_features=10000)

In [118]:
tv.fit(clean_reviewes)
tfidf = tv.transform(df['clean_review'])

In [15]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [20]:
X, y = tfidf, df['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33)
X_train.shape, y_train.shape

((16750, 500), (16750,))

In [21]:
nb = MultinomialNB()
nb.fit(X_train, y_train)

ValueError: Input X must be non-negative

In [22]:
lr = LogisticRegression()
lr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [23]:
svc = LinearSVC()
svc.fit(X_train, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [24]:
# y_pred = nb.predict(X_test)
# print 'nb = ', roc_auc_score(y_pred, y_test)
y_pred = lr.predict(X_test)
print 'lr = ', roc_auc_score(y_pred, y_test)
y_pred = svc.predict(X_test)
print 'svc = ', roc_auc_score(y_pred, y_test)

lr =  0.853669783999
svc =  0.862282871846


In [25]:
# nb.fit(X, y)
lr.fit(X, y)
svc.fit(X, y)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [26]:
# tfidf_test = tv.transform(df_test['clean_review'])
tfidf_test = compute_tfidf(df_test['clean_review'])
y_output = svc.predict(tfidf_test)
df_output = pd.DataFrame({'id': df_test['id'], 'sentiment': y_output}, columns=('id', 'sentiment'))
df_output.to_csv('submission.csv', index = False)

__try word2vec__

In [230]:
from gensim.models import word2vec

In [231]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print "Training model..."
model = word2vec.Word2Vec(clean_reviewes, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

2016-10-25 19:08:47,451 : INFO : collecting all words and their counts
2016-10-25 19:08:47,452 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types


Training model...


2016-10-25 19:08:47,704 : INFO : PROGRESS: at sentence #10000, processed 1205223 words, keeping 51374 word types
2016-10-25 19:08:47,946 : INFO : PROGRESS: at sentence #20000, processed 2396605 words, keeping 67660 word types
2016-10-25 19:08:48,190 : INFO : PROGRESS: at sentence #30000, processed 3584935 words, keeping 81517 word types
2016-10-25 19:08:48,463 : INFO : PROGRESS: at sentence #40000, processed 4782315 words, keeping 93232 word types
2016-10-25 19:08:48,808 : INFO : PROGRESS: at sentence #50000, processed 5994110 words, keeping 103318 word types
2016-10-25 19:08:49,171 : INFO : PROGRESS: at sentence #60000, processed 7199355 words, keeping 112019 word types
2016-10-25 19:08:49,449 : INFO : PROGRESS: at sentence #70000, processed 8392228 words, keeping 119672 word types
2016-10-25 19:08:49,584 : INFO : collected 123345 word types from a corpus of 8992035 raw words and 74998 sentences
2016-10-25 19:08:49,584 : INFO : Loading a fresh vocabulary
2016-10-25 19:08:49,686 : INFO

In [12]:
def words_to_vec(model, words):
    vec = np.zeros(model.vector_size)
    count = 0
    for w in words:
        # print w
        if w not in model.vocab: continue
        v = model[w]
        vec += v
        count += 1
    if count:
        vec /= count
    return vec

In [11]:
def compute_tfidf(review):
    tfidf = review.apply(lambda x: words_to_vec(model, x))
    tmp = []
    for x in tfidf:
        tmp.append(x.tolist())
    tfidf = np.array(tmp)
    return tfidf

In [18]:
tfidf = compute_tfidf(df['clean_review'])

In [19]:
tfidf_test = compute_tfidf(df_test['clean_review'])

In [215]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
# whish NB can deals with data.
tfidf = scaler.fit_transform(tfidf)

__try doc2vec__

In [17]:
from gensim.models import doc2vec
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 500    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
documents = map(lambda (idx, x): doc2vec.TaggedDocument(x, [idx]), enumerate(clean_reviewes))
print "Training model..."
model = doc2vec.Doc2Vec(documents, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

2016-10-25 19:29:35,108 : INFO : collecting all words and their counts
2016-10-25 19:29:35,109 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2016-10-25 19:29:35,246 : INFO : PROGRESS: at example #10000, processed 905662 words (6614201/s), 42242 word types, 10000 tags


Training model...


2016-10-25 19:29:35,402 : INFO : PROGRESS: at example #20000, processed 1801245 words (5764451/s), 55223 word types, 20000 tags
2016-10-25 19:29:35,560 : INFO : PROGRESS: at example #30000, processed 2695466 words (5707121/s), 65815 word types, 30000 tags
2016-10-25 19:29:35,722 : INFO : PROGRESS: at example #40000, processed 3596299 words (5575872/s), 74658 word types, 40000 tags
2016-10-25 19:29:35,886 : INFO : PROGRESS: at example #50000, processed 4506734 words (5573560/s), 82079 word types, 50000 tags
2016-10-25 19:29:36,047 : INFO : PROGRESS: at example #60000, processed 5411873 words (5638689/s), 88445 word types, 60000 tags
2016-10-25 19:29:36,207 : INFO : PROGRESS: at example #70000, processed 6309853 words (5622845/s), 94195 word types, 70000 tags
2016-10-25 19:29:36,300 : INFO : collected 96834 word types and 74998 unique tags from a corpus of 74998 examples and 6760780 words
2016-10-25 19:29:36,300 : INFO : Loading a fresh vocabulary
2016-10-25 19:29:36,399 : INFO : min_cou